In [1]:
# set up hsds server, .hscfg problem is fixed with fix in personal slack.   
import pickle
import pandas as pd
import os

import math
import numpy as np

import matplotlib.pyplot as plt
import tables
from rex import WindX
from rex import Resource
import h5pyd
import rex
from rex import MultiFileResourceX, ResourceX, MultiTimeResourceX

In [ ]:
"""
Things to do:
    1. Read coords from map_uid_lat_lon file.
    2. get two corners of the region 
         - top left is max latitude, min longitude
         - bottom right is min latitude, max longitude
    3. Somehow, find a way to get multiple years of data into one ResourceX. The nrel tutorial is vague on this.
    4. run res.ResourceX().box_gids(lat_lon_1, lat_lon_2) to get gids within the box. <- maybe run get_box_df
    5. Copy the data
    6. dfs = [pd.DataFrame(res[var, :, gid], index = ti) for gid in gids]
    7. Save to different files (or something else, idk right now).

    Rex Docs: 
    https://nrel.github.io/rex/_autosummary/rex.resource_extraction.resource_extraction.ResourceX.html#rex.resource_extraction.resource_extraction.ResourceX.box_gids
"""

In [2]:
folder = "/nrel/sup3rcc/"
with h5pyd.Folder(folder) as f:
    print(f"\n {folder}:")
    for i in range(len(list(f))):
        print(list(f)[i])



 /nrel/sup3rcc/:
conus_ecearth3_ssp585_r1i1p1f1
conus_mriesm20_ssp585_r1i1p1f1


In [3]:
# Coords data 
filename = "map_uid_lat_lon.pkl"
with open(filename, 'rb') as file:
    data = pickle.load(file)


In [4]:
# using European GCM
gcm = "ecearth3"
data_dir = "/nrel/sup3rcc/conus_ecearth3_ssp585_r1i1p1f1/"
map_uid_df = pd.DataFrame.from_dict(data, orient='index', columns=['latitude', 'longitude']).reset_index()

# Applying gid to lat, lon sites. requires ONE file to be read from
filename = os.path.join(data_dir, f"sup3rcc_conus_{gcm}_ssp585_r1i1p1f1_2020.h5")
res = ResourceX(filename, hsds=True)
map_uid_df['gid'] = res.lat_lon_gid(list(data.values()))

print(map_uid_df.head())

# Now get temps. Estimated time: > 2 hours
var = "temperature_2m"
filename = os.path.join(data_dir, f"sup3rcc_conus_{gcm}_ssp585_r1i1p1f1_202*.h5")
res = MultiTimeResourceX(filename, hsds=True)
df = res.get_gid_df(var, map_uid_df.gid.unique())

print(df.head())

outfile = "temperatures_by_gid_time.h5"
df.to_hdf(outfile, key='temps')
map_uid_df.to_hdf(outfile, key='gids')


           index   latitude  longitude     gid
0       1202_G_9  41.483776 -91.118699  799447
1       1897_B_3  47.578635 -92.514570  300298
2       1155_G_1  40.736379 -93.784580  859152
3  56427_G_Unit1  42.669880 -89.075284  700620
4       2005_G_5  45.929043 -93.632996  433658
                           799447     300298   859152   700620     433658   \
time_index                                                                   
2020-01-01 00:00:00+00:00    -7.19 -14.150000    -4.98    -6.98 -13.620000   
2020-01-01 01:00:00+00:00    -7.46 -14.910000    -5.88    -9.25 -14.580000   
2020-01-01 02:00:00+00:00    -8.45 -16.389999    -7.02   -11.16 -16.990000   
2020-01-01 03:00:00+00:00    -9.61 -18.629999    -7.75   -12.61 -18.740000   
2020-01-01 04:00:00+00:00   -10.14 -19.200001    -8.84   -13.51 -18.940001   

                             509577   401782   645304   788119   645476   ...  \
time_index                                                                ...   
2020-01-0

HDF5ExtError: HDF5 error back trace

  File "D:\bld\hdf5_1733002084650\work\src\H5F.c", line 836, in H5Fopen
    unable to synchronously open file
  File "D:\bld\hdf5_1733002084650\work\src\H5F.c", line 796, in H5F__open_api_common
    unable to open file
  File "D:\bld\hdf5_1733002084650\work\src\H5VLcallback.c", line 3863, in H5VL_file_open
    open failed
  File "D:\bld\hdf5_1733002084650\work\src\H5VLcallback.c", line 3675, in H5VL__file_open
    open failed
  File "D:\bld\hdf5_1733002084650\work\src\H5VLnative_file.c", line 128, in H5VL__native_file_open
    unable to open file
  File "D:\bld\hdf5_1733002084650\work\src\H5Fint.c", line 1960, in H5F_open
    unable to lock the file
  File "D:\bld\hdf5_1733002084650\work\src\H5FD.c", line 2411, in H5FD_lock
    driver lock request failed
  File "D:\bld\hdf5_1733002084650\work\src\H5FDsec2.c", line 941, in H5FD__sec2_lock
    unable to lock file, errno = 0, error message = 'No error', Win32 GetLastError() = 33

End of HDF5 error back

In [5]:
# Get lat_lon_1 (top_left), lat_lon_2 (bottom_right)

outfile = "temperatures_by_gid_time_!.h5"
df.to_hdf(outfile, key='temps')
map_uid_df.to_hdf(outfile, key='gids')


In [25]:
#tree = sp.KDTree(list(data.values()), copy_data=True)


OSError: Could not open file in mode "r": "/nrel/sup3rcc/conus_ecearth3_ssp585_r1i1p1f1/sup3rcc_conus_ecearth3_ssp585_r1i1p1f1_202*.h5"

In [1]:
df.to_hdf(outfile, key='temps')
map_uid_df.to_hdf(outfile, key='gids')

NameError: name 'df' is not defined

In [18]:
df.shape

(8760, 371)

In [7]:
#Opening datapandas.HDFStore(data_store)


['/gids', '/temps']


In [26]:
map_uid_df.head()

,index,latitude,longitude
0,1202_G_9,41.483776,-91.118699
1,1897_B_3,47.578635,-92.514570
2,1155_G_1,40.736379,-93.784580
3,56427_G_Unit1,42.669880,-89.075284
4,2005_G_5,45.929043,-93.632996
